<a href="https://colab.research.google.com/github/Anshusidda99/fmml/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1.
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


Answers:
1. Yes, averaging the validation accuracy across multiple splits (commonly referred to as cross-validation) can provide more consistent and reliable results in machine learning compared to a single train-validation split. Cross-validation helps to address some of the variability that may arise from the randomness in data splitting and ensures a more robust evaluation of your model's performance. It is particularly valuable when you have a limited dataset.

Here's how cross-validation works and why it's beneficial:

Data Splitting: Instead of dividing your dataset into just one training set and one validation set, cross-validation involves splitting your data into multiple subsets, typically referred to as "folds."

Multiple Training and Validation Runs: You then train and validate your model multiple times, each time using a different fold as the validation set and the remaining folds as the training set. This ensures that your model is evaluated on different portions of the data.

Averaging Results: After each run, you record the validation accuracy (or other evaluation metrics). Once all runs are complete, you calculate the average validation accuracy and often the standard deviation to assess consistency.

The benefits of cross-validation include:

Better Estimation: Cross-validation provides a more accurate estimate of your model's performance because it considers multiple scenarios of how the data can be split. This helps reduce the impact of data variability on your results.

Reduced Overfitting: It can help detect overfitting because the model is tested on multiple validation sets, making it more likely to identify if the model performs well on average or if it's overly specific to a single validation set.

Improved Robustness: Cross-validation ensures that the results are less dependent on a particular random split, making them more reliable and robust.

Common types of cross-validation include k-fold cross-validation and stratified k-fold cross-validation, where k represents the number of folds. The choice of k depends on your dataset size and the trade-off between computational resources and result stability.

In summary, using cross-validation to average validation accuracy across multiple splits is a good practice in machine learning, especially when dealing with limited data. It provides more consistent and trustworthy performance metrics for your models, enabling better model selection and hyperparameter tuning.




2. Cross-validation provides a more accurate estimate of how your machine learning model is likely to perform on unseen data, which is similar to the test accuracy. However, it's important to note that cross-validation is still an estimate and not a direct substitute for evaluating a model on a completely independent test dataset. Let's break down the relationship between cross-validation and test accuracy:

Cross-Validation: In cross-validation, your dataset is divided into multiple subsets (folds). The model is trained and evaluated multiple times, with each fold serving as a validation set while the others are used for training. This process provides a set of validation accuracy scores.

Test Accuracy: The test accuracy, on the other hand, is obtained by evaluating your trained model on a completely separate dataset that the model has never seen during training or cross-validation. This is typically a holdout dataset reserved for final evaluation.

Here's how cross-validation relates to test accuracy:

Cross-validation provides a more robust estimate of how well your model generalizes to unseen data compared to a single train-test split. By averaging validation scores across multiple folds, you reduce the influence of randomness in data splitting.

While cross-validation is a valuable tool for model assessment, the test accuracy on a truly independent dataset is the most accurate representation of how your model will perform in a real-world scenario. It helps ensure that your model hasn't overfit to the training data or to any specific fold during cross-validation.

It's common practice to use cross-validation during model development and hyperparameter tuning to get a sense of how well your model is likely to perform. Once you've finalized your model and hyperparameters, you then evaluate it on the test dataset to get the most reliable estimate of its performance.

In summary, cross-validation provides a more accurate estimate of how well your model generalizes to unseen data compared to a single train-test split. However, the truest estimate of test accuracy still comes from evaluating your model on an independent test dataset that it has never encountered before. This ensures that your model's performance is assessed on completely new and unseen examples, which is critical for assessing its real-world performance.




3. The number of iterations, often referred to as epochs in machine learning, can have a significant impact on the estimate of your model's performance. However, the relationship between the number of iterations and the quality of the estimate is not always straightforward. The effect of the number of iterations depends on several factors:

Underfitting and Convergence: Initially, as you increase the number of iterations, your model typically learns more about the training data, which can lead to better fitting of the data and improved performance. This is especially true if the model was underfitting initially, meaning it hadn't learned the data patterns well. In such cases, more iterations can help the model converge to a better solution.

Convergence Speed: Some models may converge quickly, meaning they reach a stable solution with good performance within a few epochs. Others may require more time to converge. The optimal number of iterations may vary based on the complexity of the model and the nature of the data.

Overfitting: Increasing the number of iterations beyond a certain point can lead to overfitting. Overfitting occurs when the model starts to memorize the training data, including noise and random variations, rather than learning to generalize to unseen data. In such cases, increasing the number of iterations can result in a drop in performance on validation or test data.

Early Stopping: To find the right number of iterations, it's common to monitor the model's performance on a separate validation dataset. When the validation performance stops improving and starts to degrade, it's a sign that the model may be overfitting, and you should stop training. This technique is known as early stopping.

Computational Resources: Training deep learning models with a large number of iterations can be computationally intensive and time-consuming. In practice, you may need to balance computational constraints with the need for more iterations.

Learning Rate and Optimization: The choice of learning rate and optimization algorithm also affects the convergence behavior of the model. Learning rate schedules, where the learning rate decreases over time, can be crucial for achieving convergence.

In summary, the effect of the number of iterations on the estimate of your model's performance depends on the specific problem, dataset, and model architecture. It's not always the case that more iterations lead to better estimates. It's important to monitor the model's performance on a validation dataset and be mindful of the trade-off between underfitting and overfitting. Finding the right balance between the number of iterations and model generalization is a key part of training machine learning models.




4. While increasing the number of iterations (epochs) in machine learning can help a model better learn from the available training data, it cannot fully compensate for having a very small training dataset or validation dataset. There are limitations to what increasing the number of iterations can achieve in such situations:

Overfitting: If you have a very small training dataset and you increase the number of iterations significantly, the risk of overfitting becomes more pronounced. Overfitting occurs when the model becomes too specialized in the training data and performs poorly on unseen data. More iterations can allow the model to fit the training data very closely, including noise and random variations, making it less capable of generalizing to new data.

Data Diversity: A small dataset may lack the diversity needed for the model to capture the full range of patterns and variations in the data. Increasing the number of iterations won't create new data; it will merely allow the model to see the same limited data more times, potentially leading to overfitting.

Validation Dataset Size: If your validation dataset is also very small, it can lead to unstable estimates of model performance. Increasing the number of iterations won't address the fundamental issue of having an insufficient amount of data for robust validation.

Model Complexity: If your model is very complex relative to the size of the dataset, it's more likely to overfit, even with a large number of iterations.

To address the challenges posed by a very small training dataset or validation dataset, consider the following strategies:

Data Augmentation: If possible, use data augmentation techniques to artificially increase the effective size of your training dataset. This can help the model generalize better.

Transfer Learning: If you have access to a pre-trained model on a related task or a larger dataset, you can fine-tune it on your small dataset. Transfer learning leverages knowledge learned from a larger dataset to improve performance on a smaller one.

Regularization: Apply regularization techniques like L1, L2, or dropout to mitigate overfitting when dealing with small datasets.

Collect More Data: If feasible, consider collecting additional data to increase the size of your training and validation datasets. A larger, diverse dataset can lead to better model generalization.

Use Simpler Models: In cases of extremely limited data, it may be beneficial to use simpler models that have fewer parameters to reduce the risk of overfitting.

In summary, while increasing the number of iterations can aid in model training, it cannot fully compensate for having a very small training or validation dataset. Addressing the limitations of small datasets often requires a combination of techniques, including data augmentation, regularization, transfer learning, and, ideally, obtaining more data if possible.




